<a href="https://colab.research.google.com/github/PWEI07/context_distiliation/blob/main/0418_llama3_NLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!chmod 755 -R /content/drive/MyDrive/colab_env/bin
# !mkdir Mixtral-8x7B-Instruct-v0.1-GPTQ

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
# todo modify the two paths below to where you store the virtual environment I shared with you.
%cd '/content/drive/MyDrive'
# !mkdir Mistral-7B-Instruct-v0.2
!source /content/drive/MyDrive/colab_env/bin/activate
import sys
packate_path = r'/content/drive/MyDrive/colab_env/lib/python3.10/site-packages'
sys.path.append(packate_path)

/content/drive/MyDrive


In [4]:
DATA_SET_NAME = "pwei07/cqa"

## Install package and LLM

In [5]:
import huggingface_hub
import os
from google.colab import userdata
from datasets import load_dataset

os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
huggingface_hub.login(os.environ["HF_TOKEN"], add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:

# !source /content/drive/MyDrive/colab_env/bin/activate; pip install accelerate==0.28.0



In [7]:
# !huggingface-cli download meta-llama/Meta-Llama-3-8B-Instruct --local-dir Meta-Llama-3-8B-Instruct --local-dir-use-symlinks False


In [8]:

# model_id = r'/content/drive/MyDrive/gemma-7b-it'
model_id = r'/content/drive/MyDrive/Meta-Llama-3-8B-Instruct'


# Load LLM

In [9]:

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import PeftModel, PeftConfig
import gc


In [10]:
def clean_gpu_cache():
  torch.cuda.empty_cache()
  gc.collect()

In [11]:
clean_gpu_cache()

In [12]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!nvidia-smi


Sun Apr 21 14:57:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0              26W / 300W |      2MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
# import os, torch, wandb, platform, warnings
import os, torch, platform, warnings

from datasets import load_dataset
from trl import SFTTrainer

bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

In [14]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    # device_map={"": 0},
    device_map="auto",
    torch_dtype=torch.float16

)
model.config.use_cache = True # silence the warnings. Please re-enable for inference!
# model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
# Load tokenizer

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_eos_token = True
# tokenizer.add_bos_token, tokenizer.add_eos_token
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
# for llama 3
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Get Data and Process it

In [17]:
def format_label_question(example):
  input = example['input']
  chat = [{"role": "system", "content": "Use common sense reasoning to answer question by choosing 1 from 5 choices (delimited with XML tags) and stating your reason\n"},
        {"role": "user", "content": f"<question>{input.split('Answer Choices:')[0]}<\question>\n<choices>{input.split('Answer Choices:')[-1]}<\choices>"}
    ]
  # f"Pick 1 from the 5 answers, then state the reason : {example['input'][0]}\nAnswer:"
  chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
  example["label_question"] = chat_text

  return example


# def format_label_question(example):
#     example["label_question"] = f"Question: {example['input']}\nAnswer:"
#     return example

In [18]:
def format_label_pair(example):
  input = example['input']
  # chat = {"prompt": f"Use common sense reasoning to answer question by choosing 1 from 5 choices\n<question>{input.split('Answer Choices:')[0]}<\question>\n<choices>{input.split('Answer Choices:')[-1]}<\choices>",
  #        "completion": example['label']}

  # chat = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False)
  # chat = f"""prompt: Use common sense reasoning to answer question by choosing 1 from 5 choices\n<question>{input.split('Answer Choices:')[0]}<\question>\n<choices>{input.split('Answer Choices:')[-1]}<\choices>\ncompletion: {example['label']}"""
  chat = f'''Below is an instruction that describes a task, paired with an input. Write a response that appropriately completes the request.

### Instruction:
Use common sense reasoning to answer question by choosing 1 from 5 choices
### Input:
<question>{input.split('Answer Choices:')[0]}<\question>\n<choices>{input.split('Answer Choices:')[-1]}<\choices>
### Response:
{example['label']}
'''
  example["label_pair"] = chat

  return example


# def format_label_pair(example):
#     example['label_pair'] = f"Question: {example['input']}\nAnswer: {example['label']}"
#     return example

In [19]:
def format_rationale_question(example):
  input = example['input']
  chat = [
        {"role": "user", "content": f"Given a question, choices, and answer, state the reason for the answer\n<question>{input.split('Answer Choices:')[0]}<\question>\n<choices>{input.split('Answer Choices:')[-1]}<\choices>\n<\question>\n<answer>{example['llm_label']}<\answer>"}
    ]
  chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
  example["rationale_question"] = chat_text

  return example

# def format_rationale_question(example):
#     example["rationale_question"] = f"Question: {example['input']}\nAnswer: {example['llm_label']}\nAnswer's reasoning:"
#     return example

In [20]:
def format_rationale_pair(example):
  input = example['input']
  # chat = [
  #       {"role": "user", "content": f"Given a question, choices, and answer, state the reason for the answer\n<question>{input.split('Answer Choices:')[0]}<\question>\n<choices>{input.split('Answer Choices:')[-1]}<\choices>\n<\question>\n<answer>{example['llm_label']}<\answer>"},
  #       {"role": "assistant", "content": example['rationale']}
  #   ]
  chat = f'''Below is an instruction that describes a task, paired with an input. Write a response that appropriately completes the task.
### Instruction:
Given question, choices, and answer (all delimited with XML tags), state the reason for the answer
### Input:
<question>{input.split('Answer Choices:')[0]}<\question>\n<choices>{input.split('Answer Choices:')[-1]}<\choices>\n<answer>{example['llm_label']}<\answer>
### Response:
{example['rationale']}
'''
  example["rationale_pair"] = chat
  # chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False)
  # example["rationale_pair"] = chat_text

  return example

# def format_rationale_pair(example):
#     example["rationale_pair"] = f"Question: {example['input']}\nAnswer: {example['llm_label']}\nAnswer's reasoning: {example['rationale']}"
#     return example

In [21]:
def get_dataset(dataset_name=DATA_SET_NAME):
  data = load_dataset(dataset_name, use_auth_token=True)
  data = data.map(format_label_question)
  data = data.map(format_label_pair)
  data = data.map(format_rationale_question)
  data = data.map(format_rationale_pair)
  return data


In [22]:
def format_label_question_anli1(example):
  input = example['input']
  chat = [{"role": "system", "content": "As a natural language inference expert, choose 1 from these 3: 'contradiction', 'entailment', 'neutral' that best describes the relationship between premise and hypothesis, then state your reason"},
          {"role": "user", "content": f"<premise>{input.split('</s>')[0]}</premise>\n<hypothesis>{input.split('</s>')[-1]}</hypothesis>"}]
  # chat = [
  #       {"role": "user", "content": f"Given a premise and a hypothesis (delimited with XML "
  #                                     f"tags). Choose 1 from these 3: 'contradiction', 'entailment', 'neutral' based "
  #                                     f"on inference relationship between premise and hypothesis and state your reason\n<premise>{input.split('</s>')[0]}</premise>\n<hypothesis>{input.split('</s>')[-1]}</hypothesis>"},
  #   ]
  chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
  example["label_question"] = chat_text
  return example

In [23]:
def format_label_pair_anli1(example):
  input = example['input']
  chat = {"prompt": f"As a natural language inference expert, choose 1 from these 3: 'contradiction', 'entailment', 'neutral' that best describes the relationship between premise and hypothesis",
         "completion": example['label']}
  # chat = [
  #       {"role": "user", "content": "You will be provided with a premise and then a hypothesis (delimited with XML "
  #                                     "tags). Choose 1 from these 3: 'contradiction', 'entailment', 'neutral' based "
  #                                     f"on inference relationship between premise and hypothesis and state your reason<premise>{input.split('</s>')[0]}</premise>\n<hypothesis>{input.split('</s>')[-1]}</hypothesis>"},
  #       {"role": "assistant", "content": example['label']}
  #   ]
  # chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False)
  example["label_pair"] = chat

  return example

In [24]:
def format_rationale_question_anli1(example):
  input = example['input']
  chat = [
        {"role": "user", "content": f"Given a premise, a hypothesis, and relationship between them (delimited with XML tags). State the reason for this relationship\n<premise>{input.split('</s>')[0]}</premise>\n<hypothesis>{input.split('</s>')[-1]}</hypothesis>\n<relationship>{example['label']}</relationship>"},
    ]
  chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
  example["rationale_question"] = chat_text

  return example

In [25]:
def format_rationale_pair_anli1(example):
  input = example['input']
  chat = [
        {"role": "user", "content": f"Given a premise, a hypothesis, and relationship between them (delimited with XML tags). State the reason for this relationship\n<premise>{input.split('</s>')[0]}</premise>\n<hypothesis>{input.split('</s>')[-1]}</hypothesis>\n<relationship>{example['llm_label']}</relationship>"},
        {"role": "assistant", "content": example['rationale']}
    ]
  chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False)

  example["rationale_pair"] = chat_text
  return example

In [26]:
def get_dataset_anli1(dataset_name=DATA_SET_NAME):
  data = load_dataset(dataset_name, use_auth_token=True)
  data = data.map(format_label_question_anli1)
  data = data.map(format_label_pair_anli1)
  data = data.map(format_rationale_question_anli1)
  data = data.map(format_rationale_pair_anli1)
  return data

In [27]:
data = get_dataset()
# print(data['train'][:2])

/content/drive/MyDrive/colab_env/lib/python3.10/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


In [28]:
data['train'][0]

{'input': 'A kid is studying new words at home in the kitchen, where could a dictionary be?\nAnswer Choices:\n(a) library\n(b) table\n(c) bookstore\n(d) shelf\n(e) classroom',
 'label': 'table',
 'llm_label': 'library',
 'rationale': 'The answer must be a place where a dictionary can be found. Of the above choices, only a library and a classroom have dictionaries.',
 'label_question': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nUse common sense reasoning to answer question by choosing 1 from 5 choices (delimited with XML tags) and stating your reason<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n<question>A kid is studying new words at home in the kitchen, where could a dictionary be?\n<\\question>\n<choices>\n(a) library\n(b) table\n(c) bookstore\n(d) shelf\n(e) classroom<\\choices><|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
 'label_pair': 'Below is an instruction that describes a task, paired with an input. Write a response that appropriat

In [29]:
%%capture
data_anli1 = get_dataset_anli1("pwei07/anli1")
# data_esnli = get_dataset_anli1("pwei07/esnli")

# data_svamp = get_dataset("pwei07/svamp")

# Evaluate Palm Accuracy


In [30]:
def evaluate_prediction(dataset, evaluation=False, pred_col='llm_label'):

    # Convert the dataset to a pandas DataFrame
    df = dataset.to_pandas()
    def safe_evaluation(row):
      try:
          return eval(str(row['label'])) == eval(str(row[pred_col]))
      except:
          return False
    # Check if 'label' matches 'llm_label'
    if not evaluation:
      df['is_correct'] = df.apply(lambda x: x['label'].lower() in x[pred_col].lower(), axis=1)
    else:
    # Apply the function across the DataFrame rows
      df['is_correct'] = df.apply(safe_evaluation, axis=1)

    # Calculate the accuracy as the percentage of correct predictions
    accuracy = df['is_correct'].mean() * 100

    # Extract rows where the condition does not hold
    incorrect_predictions = df[~df['is_correct']]

    # Return accuracy and the incorrect predictions DataFrame
    return accuracy, incorrect_predictions

In [31]:
palm_accuracy_cqa = evaluate_prediction(data['valid'], evaluation=False)
palm_accuracy_anli1 = evaluate_prediction(data_anli1['valid'], evaluation=False)
# palm_accuracy_esnli = evaluate_prediction(data_esnli['valid'], evaluation=False)
# palm_accuracy_svamp = evaluate_prediction(data_svamp['valid'], evaluation=True)

In [32]:

print("palm_accuracy_cqa", palm_accuracy_cqa[0])
print("palm_accuracy_anli1", palm_accuracy_anli1[0])
# print("palm_accuracy_esnli", palm_accuracy_esnli[0])
# print("palm_accuracy_svamp", palm_accuracy_svamp[0])


palm_accuracy_cqa 76.0
palm_accuracy_anli1 66.8


# test LLM without chat template

In [33]:
def get_llm_answer(text, max_new_tokens=None):
  temp_encoding = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True).to('cuda')
  temp_token_id = temp_encoding['input_ids']
  # temp_attention_mask = temp_encoding['attention_mask'].to(device)
  # print(temp_attention_mask)
  # outputs = model.generate(input_ids=temp_token_id.unsqueeze(0), attention_mask=temp_attention_mask)
  outputs = model.generate(**temp_encoding, eos_token_id=terminators, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
  # print(temp_token_id.shape)
  # print(outputs)
  return tokenizer.batch_decode(outputs[:,temp_token_id.shape[1]:], skip_special_tokens=True)[0]

In [34]:
# example = data['train'][:1]
# text_temp = f"Pick 1 from the 5 answers, then state the reason : {example['input'][0]}\nAnswer:"
# print(text_temp)
# print(get_llm_answer(text_temp,max_new_tokens=30))

In [35]:
# text_temp = f"Question: {example['input'][0]}\nAnswer: {example['label'][0]}\nAnswer's reasoning:"
# get_llm_answer(text_temp, max_new_tokens=64)


# test LLM with chat template

In [36]:
def apply_chat_template(text):
  chat = [
    {"role": "user", "content": text},
    # {"role": "assistant", "content": "The sun."}
  ]
  chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
  # print(chat_text)
  chat_text = tokenizer.apply_chat_template(chat, tokenize=True, add_generation_prompt=True, return_tensors="pt")
  # print(chat_text)
  return chat_text

In [37]:
def get_llm_answer_w_chat_template(text,  max_new_tokens=8):
  inputs = apply_chat_template(text)
  outputs = model.generate(inputs, max_new_tokens=max_new_tokens)
  # print(outputs.logits.shape)
  return tokenizer.batch_decode(outputs[:,inputs.shape[1]:], skip_special_tokens=True)[0]

In [38]:
# text_temp = f"Question: {example['input'][0]}\nAnswer:"
get_llm_answer(data_anli1['train'][0]['label_question'], max_new_tokens=16)
# get_llm_answer_w_chat_template(text_temp,  max_new_tokens=8)

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


'I would choose "contradiction" to describe the relationship between the premise and'

In [39]:
data_anli1['train'][0]

{'label': 'entailment',
 'llm_label': 'entailment',
 'rationale': 'The premise mentions that the system has 4 urban routes.',
 'input': 'The Parma trolleybus system (Italian: "Rete filoviaria di Parma" ) forms part of the public transport network of the city and "comune" of Parma, in the region of Emilia-Romagna, northern Italy. In operation since 1953, the system presently comprises four urban routes.</s>The trolleybus system has over 2 urban routes',
 'label_question': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nAs a natural language inference expert, choose 1 from these 3: \'contradiction\', \'entailment\', \'neutral\' that best describes the relationship between premise and hypothesis, then state your reason<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n<premise>The Parma trolleybus system (Italian: "Rete filoviaria di Parma" ) forms part of the public transport network of the city and "comune" of Parma, in the region of Emilia-Romagna, northern Italy. In

In [40]:
get_llm_answer(data['train'][0]['label_question'],max_new_tokens=32)


"<answer>(d) shelf</answer>\n\nReason: Since the kid is studying new words at home in the kitchen, it's likely that the dictionary is placed"

In [41]:
# get_llm_answer(data_esnli['train'][0]['rationale_question'],max_new_tokens=32)
data_anli1['train'][0]

{'label': 'entailment',
 'llm_label': 'entailment',
 'rationale': 'The premise mentions that the system has 4 urban routes.',
 'input': 'The Parma trolleybus system (Italian: "Rete filoviaria di Parma" ) forms part of the public transport network of the city and "comune" of Parma, in the region of Emilia-Romagna, northern Italy. In operation since 1953, the system presently comprises four urban routes.</s>The trolleybus system has over 2 urban routes',
 'label_question': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nAs a natural language inference expert, choose 1 from these 3: \'contradiction\', \'entailment\', \'neutral\' that best describes the relationship between premise and hypothesis, then state your reason<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n<premise>The Parma trolleybus system (Italian: "Rete filoviaria di Parma" ) forms part of the public transport network of the city and "comune" of Parma, in the region of Emilia-Romagna, northern Italy. In

# Test the untuned model's performance on validation set

In [42]:
def evaluate_model_accuracy(data, n=50, chat=False, max_new_tokens=30, evaluation=False):
  pred = []
  for i in range(len(data)):
    if not chat:
      pred.append(get_llm_answer(data[i]['label_question'],  max_new_tokens=max_new_tokens))
    else:
      pred.append(get_llm_answer_w_chat_template(data[i]['label_question'], max_new_tokens=max_new_tokens))
    if i > n:
      break
  data_w_pred = data.select(range(len(pred))).add_column("pred", pred)
  accuracy, inaccurate_pred = evaluate_prediction(data_w_pred, evaluation=evaluation, pred_col='pred')
  return accuracy, inaccurate_pred, pred

In [43]:
def get_label_accuracy(pred, gt):
  # gt stands for ground truth
  correct_count = 0
  wrong_idx = []
  for k, (i,j) in enumerate(zip(pred, gt)):
    if j.lower() in i.lower(9):
      correct_count += 1
    else:
      wrong_idx.append(k)
  return correct_count / len(pred), wrong_idx

In [44]:
import warnings
import re

# To ignore specific warnings with regular expression patterns:
warnings.filterwarnings('ignore', message=r".*pad_token_id.*")
warnings.filterwarnings('ignore', message=r".*decoder-only architecture.*")

# Your code here, e.g., using a model from the transformers library


In [ ]:
# acc = get_label_accuracy(vali_label_pred, datac['label'][:len(vali_label_pred)])
# acc_chat = get_label_accuracy(vali_label_w_chat_pred, data['valid']['label'][:len(vali_label_pred)])
# print(acc)
# print(acc_chat)

%%capture
cqa_model_acc = evaluate_model_accuracy(data['valid'], n=100, chat=False, max_new_tokens=30, evaluation=False)
# cqa_model_chat_acc = evaluate_model_accuracy(data['valid'], n=50, chat=True, max_new_tokens=30, evaluation=False)
print(cqa_model_acc[0])
# print(cqa_model_chat_acc[0])`

In [ ]:
# print(cqa_model_acc[0])
# print(cqa_model_chat_acc[0])
print(cqa_model_acc[0])
cqa_model_acc[1].to_csv('/content/drive/MyDrive/llm_output/cqa_wrong_llama3_.csv')

70.58823529411765


In [ ]:
anli1_model_acc = evaluate_model_accuracy(data_anli1['valid'], n=100, chat=False, max_new_tokens=30, evaluation=False)
# anli1_model_chat_acc = evaluate_model_accuracy(data_anli1['valid'], n=50, chat=True, max_new_tokens=30, evaluation=False)

In [ ]:
anli1_model_acc[0]

55.88235294117647

In [ ]:

anli1_model_acc[1].to_csv('/content/drive/MyDrive/llm_output/anli1_wrong_llama3.csv')

In [ ]:
# esnli_model_acc = evaluate_model_accuracy(data_esnli['valid'], n=50, chat=False, max_new_tokens=30, evaluation=False)

# print(esnli_model_acc[0])
# esnli_model_acc[1].to_csv('/content/drive/MyDrive/esnli_wrong_mistral.csv')

In [ ]:
# print(esnli_model_acc[0])
# # esnli_model_acc[1]

42.30769230769231


# FT on label

In [45]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
# import os, torch, wandb, platform, warnings
import os, torch, platform, warnings

from datasets import load_dataset
from trl import SFTTrainer

peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",]
    )


In [46]:
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1

In [47]:

model.gradient_checkpointing_enable()
model = get_peft_model(model, peft_config)
model = prepare_model_for_kbit_training(model)

# Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_eos_token = True
# tokenizer.add_bos_token, tokenizer.add_eos_token


In [48]:
# Training Arguments
# Hyperparameters should beadjusted based on the hardware you using
new_model = "llama3_ft_0420" #set the name of the new model
output_dir = r"/content/drive/MyDrive/llama3_ft_0421"


training_arguments = TrainingArguments(
    output_dir= output_dir,
    num_train_epochs=2,
    gradient_checkpointing=True,
    # max_steps=200,
    # auto_find_batch_size=True,
    per_device_train_batch_size= 2,
    gradient_accumulation_steps= 4,
    optim = "paged_adamw_8bit",
    save_steps= 100,
    logging_steps= 100,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    # max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    save_total_limit = 1,
    load_best_model_at_end=True,
    evaluation_strategy='steps',
    save_strategy='steps',
    resume_from_checkpoint=True,
    lr_scheduler_type= "cosine",
    per_device_eval_batch_size=2,
    eval_accumulation_steps=2,
    prediction_loss_only=True,
    # report_to="wandb"
)

In [49]:
# model.config.use_cache = True # silence the warnings. Please re-enable for inference!
# get_llm_answer(data['valid'][200]['rationale_question'],max_new_tokens=32)


In [50]:
# model.config.use_cache = True # silence the warnings. Please re-enable for inference!

# get_llm_answer(data['valid'][6]['label_question'],max_new_tokens=32)


In [ ]:
model.config.use_cache = False # silence the warnings. Please re-enable for inference!

trainer_label = SFTTrainer(
    model=model,
    train_dataset=data['train'],
    eval_dataset=data['valid'],
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="label_pair",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= True,
    # formatting_func=formatting_prompts_func,

)
trainer_label.train()
model.config.use_cache = True # silence the warnings. Please re-enable for inference!


/content/drive/MyDrive/colab_env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/content/drive/MyDrive/colab_env/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maint

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


In [ ]:
# clean_gpu_cache()
%%capture
cqa_model_acc = evaluate_model_accuracy(data['valid'], n=50, chat=False, max_new_tokens=30, evaluation=False)
# cqa_model_chat_acc = evaluate_model_accuracy(data['valid'], n=50, chat=True, max_new_tokens=30, evaluation=False)
print(cqa_model_acc[0])


In [ ]:
print(cqa_model_acc[0])


In [ ]:
model.save_pretrained("/content/drive/MyDrive/llama3_label_pretrained")

# Below is still in development, no need to run

# try to save and load adapter

In [ ]:
trainer_label.save_model('/content/drive/MyDrive/trainer_label_llama3')

/content/drive/MyDrive/colab_env/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /content/drive/MyDrive/Meta-Llama-3-8B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(


In [ ]:
model.set_adapter('/content/drive/MyDrive/trainer_label_llama3')
model.config.use_cache = True # silence the warnings. Please re-enable for inference!
model

In [ ]:
model.active_adapter()

get_llm_answer(data['valid'][6]['label_question'],max_new_tokens=32)

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


"<(c) purse>\n\nI would choose (c) purse because it's common sense to keep your car keys in a place where you can easily access them when"

In [ ]:
model.disable_adapters()
get_llm_answer(data['valid'][6]['label_question'],max_new_tokens=32)

"<(c) purse>\n\nI would choose (c) purse because it's common sense to keep your car keys in a place where you can easily access them when"

In [ ]:
# del model  # Delete the model to decrease reference count
# torch.cuda.empty_cache()

# Load the adapter layer

In [ ]:
from peft import AutoPeftModelForCausalLM

peft_model_id = "/content/drive/MyDrive/trainer_label_llama3"
# peft_model_id = args.output_dir

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  quantization_config=bnb_config,
  device_map={"": 0}
  # device_map="auto",
  # torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model

In [ ]:

# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=data['train'],
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="rationale_pair",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
    # formatting_func=formatting_prompts_func,

)
print(len(data['train']))
clean_gpu_cache()

/content/drive/MyDrive/colab_env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/8766 [00:00<?, ? examples/s]

/content/drive/MyDrive/colab_env/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


8766


In [ ]:
trainer.train()
# Save the fine-tuned model

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
trainer.save_model('/content/drive/MyDrive/trainer_rationale_llama3')

In [ ]:
model.add_adapter(peft_config, '/content/drive/MyDrive/trainer_rationale_llama3')

In [ ]:
model.add_adapter(peft_config, '/content/drive/MyDrive/trainer_label_llama3')

In [ ]:
model.enable_adapters()


In [ ]:
model

In [ ]:
peft_model_id = "/content/drive/MyDrive/trainer_rationale_llama3"
# peft_model_id = args.output_dir

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  quantization_config=bnb_config,
  device_map={"": 0}
  # device_map="auto",
  # torch_dtype=torch.float16
)

OSError: None is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
%%capture
cqa_model_acc = evaluate_model_accuracy(data['valid'], n=10, chat=False, max_new_tokens=30, evaluation=False)
# cqa_model_chat_acc = evaluate_model_accuracy(data['valid'], n=50, chat=True, max_new_tokens=30, evaluation=False)
print(cqa_model_acc[0])

In [ ]:
print(cqa_model_acc[0])

75.0


In [ ]:
trainer.save_model()


In [ ]:
# from peft import AutoPeftModelForCausalLM

# # Load PEFT model on CPU
# model = AutoPeftModelForCausalLM.from_pretrained(
#     args.output_dir,
#     torch_dtype=torch.float16,
#     low_cpu_mem_usage=True,
# )
# # Merge LoRA and base model and save
# merged_model = model.merge_and_unload()
# merged_model.save_pretrained(args.output_dir,safe_serialization=True, max_shard_size="2GB")

In [ ]:
trainer.model.save_pretrained(new_model)


In [ ]:
## use the below one to shut down kernal automatically to save credit


from google.colab import runtime
runtime.unassign()